In [1]:
pip install -e .

Obtaining file:///home/ec2-user/SageMaker/nano-graphrag
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 145.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 117.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 109.3 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=183330 sha256=9a142c6753ce85c4

In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
!curl https://raw.githubusercontent.com/gusye1234/nano-graphrag/main/tests/mock_data.txt > ./book.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0  1901k      0 --:--:-- --:--:-- --:--:-- 1905k


In [11]:
import os
import time
import json
import boto3
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Ensure correct AWS Region
os.environ["AWS_REGION_NAME"] = "us-east-1"

logger.info("✅ Environment setup complete. Proceeding with AWS authentication.")

INFO:__main__:✅ Environment setup complete. Proceeding with AWS authentication.


In [12]:
# Read IAM role for cross-account access
ROLE_TO_ASSUME = Path(os.path.join(os.environ["HOME"], "BedrockCrossAccount.txt")).read_text().strip()
logger.info(f"ROLE_TO_ASSUME={ROLE_TO_ASSUME}")

# Use AWS STS to assume role
credentials = boto3.client("sts").assume_role(
    RoleArn=ROLE_TO_ASSUME,
    RoleSessionName="NanoGraphRAGSession"
)["Credentials"]

# Set temporary AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = credentials["AccessKeyId"]
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials["SecretAccessKey"]
os.environ["AWS_SESSION_TOKEN"] = credentials["SessionToken"]

logger.info("✅ Successfully assumed AWS role. Proceeding with validation.")

INFO:__main__:ROLE_TO_ASSUME=arn:aws:iam::605134468121:role/BedrockCrossAccount
INFO:__main__:✅ Successfully assumed AWS role. Proceeding with validation.


In [13]:
# Verify the IAM role assigned to the notebook
!aws sts get-caller-identity

{
    "UserId": "AROAYZZGTGQMSXMXAZQY7:NanoGraphRAGSession",
    "Account": "605134468121",
    "Arn": "arn:aws:sts::605134468121:assumed-role/BedrockCrossAccount/NanoGraphRAGSession"
}


In [14]:
from nano_graphrag import GraphRAG, QueryParam

In [15]:
# Initialize GraphRAG with Amazon Bedrock
graph_func = GraphRAG(
    working_dir="./data",
    using_amazon_bedrock=True,
    best_model_id="us.anthropic.claude-3-sonnet-20240229-v1:0",
    cheap_model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
)

logger.info("✅ GraphRAG initialized with Amazon Bedrock.")

INFO:nano-graphrag:Switched the default openai funcs to Amazon Bedrock
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 0 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-graphrag:Loaded graph from ./data/graph_chunk_entity_relation.graphml with 0 nodes, 0 edges
INFO:nano-vectordb:Load (0, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './data/vdb_entities.json'} 0 data
INFO:__main__:✅ GraphRAG initialized with Amazon Bedrock.


In [16]:
async def insert_data():
    with open("./book.txt") as f:
        await graph_func.ainsert(f.read())  # Asynchronous insert

await insert_data()  # Run in Jupyter Notebook

INFO:nano-graphrag:[New Docs] inserting 1 docs
INFO:nano-graphrag:[New Chunks] inserting 42 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:nano-graphrag:Writing graph with 0 nodes, 0 edges


AccessDeniedException: An error occurred (AccessDeniedException) when calling the Converse operation: User: arn:aws:sts::257389450945:assumed-role/LabRole/SageMaker is not authorized to perform: bedrock:InvokeModel on resource: arn:aws:bedrock:us-east-1:257389450945:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0 because no identity-based policy allows the bedrock:InvokeModel action